# Mini Project 1 Problem 1: Least Squares Classifier #
### Name: Fei Yin ### 
### PID: A15555426 ###

In this problem, we will first solve a least squares problem. Using its solution, we will build a binary classifier, a one-versus-all classifier, and a one-versus-one classifier to classify hand-written digits. Finally, we will evaluate the classifiers' performances.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from numpy.linalg import pinv

In [2]:
## Load data from mnist.mat
mnist = loadmat('mnist.mat')
train_x = mnist['trainX']
train_y = mnist['trainY']
test_x = mnist['testX']
test_y = mnist['testY']

In [3]:
## Filter out pixels with nonzero values in < 600 training samples
pixel_nonzero_count = np.zeros(28 * 28)

for image in train_x:
    pixel_nonzero_count = np.where(image != 0, pixel_nonzero_count + 1, pixel_nonzero_count)

train_x = train_x[:, pixel_nonzero_count >= 600]
test_x = test_x[:, pixel_nonzero_count >= 600]

In [4]:
## Verify shapes of training and test data
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(60000, 493)
(1, 60000)
(10000, 493)
(1, 10000)


## Solving the Least Squares Problem ##
To build the binary classifier, one-versus-all classifier, and one-versus-one classifier, we must first solve the least squares problem, which is given as $\min_{\beta, \alpha}\sum_{i=1}^N (y_i - \beta^T x_i - \alpha)^2$. This is equivalent to $\min_{\beta, \alpha} ||y - \beta^T x - \mathrm{A}||^2$ where $\mathrm{A}$ is a list of $\alpha$'s and has the same shape as $y$. 

Before doing anything else, we first want to check the dimension of this problem. Since the given y (`train_y`) has shape (1, 60000), x (`train_x`) has shape (60000, 493), and we want $\beta$ to have 493 elements, we need to transpose x to get it into shape (493, 60000) where $y - \beta^T x^T$ could have the correct dimension.

In order to convert this problem into the form $\min_{x} ||y - Ax||^2$, which we can solve by solving the normal equation, we would have to 1) absorb $\mathrm{A}$ into $\beta^T x^T$ and 2) reverse the order of $\beta^T$ and $x^T$ because in our problem, $x^T$ is the feature vector, and $\beta^T$ is the coefficient vector we want to solve for. 

1) To absorb $\mathrm{A}$ into $\beta^T x^T$, we can add $\alpha$ to the end of $\beta$, making $\beta$ a 494-vector, then add a column of 1's to x, such that the problem $y_i - \beta^T x_i - \alpha$ can be preserved.

2) To reverse the order of $\beta^T$ and $x^T$ in $\beta^T x^T$, we first convert $\beta^T x^T$ to $(x\beta)^T$. Then, we convert $y - (x\beta)^T$ to $(y^T - x\beta)^T$, which are all done by utilizing the identities of tranpose. Note that the outer-most transpose can be ignored because it does not interfere with the minimization problem.

Finally, we have our problem in the form $\min_{\beta} ||y^T - x\beta||^2$, which we can solve by calculating $\beta = (x^T x)^{-1} x^T y^T$. This is implemented in the `solveLeastSquares` function below.

In [5]:
def solveLeastSquares(features, labels):
    """Solve a least squares problem.

    Keyword arguments:
    features -- a (m x n) matrix with m features each as an n-vector
    labels -- a (1 x m) matrix with m labels 
    
    Returns:
    B -- a (n x 1) matrix with n trained coefficients 
    a -- a number representing the trained bias
    """
    
    ## Add a column of ones to features to account for the bias term a
    features_new = np.column_stack((features, np.ones(features.shape[0])))
    
    ## Solve the least square problem to get the minimizer B and bias term a
    B_and_a = pinv(features_new.T @ features_new) @ features_new.T @ labels.T
    B = B_and_a[0:-1]
    a = B_and_a[-1]
    
    return B, a

With `solveLeastSquares`, we can now implement the `binaryClassifier` and `oneVersusAllClassifier` functions. The least squares problems are always solved with training features and training labels whereas predictions are made with training features in the case of training or testing features in the case of testing. **Note that the functions assume testing case if testing features are included in the functions' parameters.**

In [6]:
def binaryClassifier(train_features, train_labels, test_features=None):
    """A binary classifier that classifies the features into two categories.

    Keyword arguments:
    train_features -- a (m x n) matrix with m training features each as an n-vector
    train_labels -- a (1 x m) matrix with m training labels
    test_features -- a (k x n) matrix with k testing features each as an n-vector
    
    Returns:
    predictions -- a (1 x m) matrix with m predictions in the case of training 
                   or a (1 x k) matrix with k predictions in the case of testing 
    """
    
    B, a = solveLeastSquares(train_features, train_labels)
    
    ## Classify the results
    if test_features is None:
        predictions = np.sign(B.T @ train_features.T + a)
    else:
        predictions = np.sign(B.T @ test_features.T + a)
     
    predictions[predictions == 0] = 1
    return predictions

In [7]:
def oneVersusAllClassifier(train_features, train_labels, test_features=None):
    """A one-versus-all classifier that classifies the features into 10 categories.

    Keyword arguments:
    train_features -- a (m x n) matrix with m training features each as an n-vector
    train_labels -- a (1 x m) matrix with m training labels
    test_features -- a (k x n) matrix with k testing features each as an n-vector
    
    Returns:
    predictions -- a (1 x m) matrix with m predictions in the case of training 
                   or a (1 x k) matrix with k predictions in the case of testing 
    """
    
    is_training = (test_features is None)
    
    if is_training:
        g_max = np.full((1, train_features.shape[0]), np.NINF)
        predictions = np.zeros((1, train_features.shape[0]))
    else:
        g_max = np.full((1, test_features.shape[0]), np.NINF)
        predictions = np.zeros((1, test_features.shape[0]))
        
    
    for digit in range(0, 10):
        
        ## Create a new set of labels for each digit
        train_labels_new = np.copy(train_labels[0]).astype(float)
        train_labels_new = np.where(train_labels_new == digit, 1, -1)
        train_labels_new = np.reshape(train_labels_new, (1, len(train_labels_new)))
        
        B, a = solveLeastSquares(train_features, train_labels_new)

        ## Calculate g and classify each feature as the digit corresponding to the largest g
        if is_training:
            g = B.T @ train_features.T + a
        else:
            g = B.T @ test_features.T + a
            
        predictions = np.where(g > g_max, digit, predictions)
        g_max = np.where(g > g_max, g, g_max)
    
    return predictions

The `oneVersusOneClassifier` function below depends on the `binaryClassifier` function. We apply `binaryClassifier` once for each pair of digits. However, note that although the coefficients are obtained with pruned sets of training features and training labels, the trained coefficients are not applied to only the pruned training_features, but all 60000 training features in the case of training or all 10000 testing features in the case of testing.   

In [8]:
def oneVersusOneClassifier(train_features, train_labels, test_features=None):
    """A one-versus-one classifier that classifies the features into 10 categories.
           This function uses the binaryClassifier function. 

    Keyword arguments:
    train_features -- a (m x n) matrix with m training features each as an n-vector
    train_labels -- a (1 x m) matrix with m training labels
    test_features -- a (k x n) matrix with k testing features each as an n-vector
    
    Returns:
    predictions -- a (1 x m) matrix with m predictions in the case of training 
                   or a (1 x k) matrix with k predictions in the case of testing 
    """
    
    is_training = (test_features is None)
    
    if is_training:
        votes = np.zeros((train_features.shape[0], 10))
    else:
        votes = np.zeros((test_features.shape[0], 10))

    for i in range(10):
        for j in range(i + 1, 10):
                        
            if is_training:
                used_indices = np.arange(train_features.shape[0])
            
            else:
                used_indices = np.arange(test_features.shape[0])
                
            ## Create copies of training features and training labels to be pruned
            train_features_new = np.copy(train_features)
            train_labels_new = np.copy(train_labels[0]).astype(float)

            ## Remove the features and labels that do not correspond to the current pair of digits
            ##   while keeping track of their original indices
            which_entries_to_keep = np.logical_or(train_labels_new == i, train_labels_new == j)
            train_features_new = train_features_new[which_entries_to_keep]
            train_labels_new = train_labels_new[which_entries_to_keep]
                
            ## Modify the training labels according to the current pair of digits
            train_labels_new[train_labels_new == i] = 1
            train_labels_new[train_labels_new == j] = -1
            train_labels_new = np.reshape(train_labels_new, (1, len(train_labels_new)))
            
            if is_training:
                ## In the case of training, the original training features are passed as testing features into
                ##   binaryClassifier so that the trained coefficients are applied to all training features 
                predictions = binaryClassifier(train_features_new, train_labels_new, train_features)
            else:
                predictions = binaryClassifier(train_features_new, train_labels_new, test_features)
                
            ## Track the votes for each digit
            for k, used_index in enumerate(used_indices):
                if predictions[0][k] == 1:
                    votes[used_index][i] += 1
                elif predictions[0][k] == -1:
                    votes[used_index][j] += 1

    return np.argmax(votes, axis=-1).reshape((1, votes.shape[0]))    

To study the performances of the classifiers, confusion matrices can be built with the `buildConfusionMatrix` function. The error rate of a confusion matrix can be calculated with the `calculateErrorRate` function given a confusion matrix as input.

In [9]:
def buildConfusionMatrix(labels, predictions):
    """Build an 11 x 11 confusion matrix for the results of oneVersusAllClassifier and oneVersusOneClassifier.

    Keyword arguments:
    labels -- a (1 x m) matrix with each entry representing a label
    predictions -- a (1 x m) matrix with each entry representing a prediction
    
    Returns:
    confusion -- a (11 x 11) confusion matrix 
    """
    
    confusion = np.zeros((11, 11))

    for label, prediction in zip(labels[0], predictions[0]):        
        confusion[int(label), int(prediction)] += 1
    
    for i in range(10):
        confusion[i, -1] = np.sum(confusion[i, 0:10])
        confusion[-1, i] = np.sum(confusion[0:10, i])
    
    confusion[-1, -1] = labels.shape[1]        
    return confusion

In [10]:
def calculateErrorRate(confusion):
    """Calculate the error rate based on a confusion matrix

    Keyword arguments:
    confusion -- a (k x k) confusion matrix, where k-1 is the number of classified categories
    
    Returns:
    error_rates -- The error rates of the classification on individual digits
                   and the average error rate of the classification on all digits
    """
    
    error_rates = []
    true_positive_count = 0
    class_count = confusion.shape[0] - 1
    
    for i in range(class_count):
        error_rates.append((confusion[i, -1] - confusion[i, i]) / confusion[i, -1])
        true_positive_count += confusion[i, i]
        
    error_rates.append((confusion[-1, -1] - true_positive_count) / confusion[-1, -1])    
    return error_rates

## One-versus-all Classifier on Training Data ##

In [11]:
## Run one-versus-all classifier on the training data
one_v_all_predictions = oneVersusAllClassifier(train_x, train_y)

In [12]:
## Build confusion matrix for the results to the one-versus-all classifier
np.set_printoptions(suppress=True)
np.set_printoptions( linewidth=100)
one_v_all_confusion = buildConfusionMatrix(train_y, one_v_all_predictions)
print(one_v_all_confusion)

[[ 5669.     8.    21.    19.    25.    46.    65.     4.    60.     6.  5923.]
 [    2.  6543.    36.    17.    20.    30.    14.    14.    60.     6.  6742.]
 [   99.   278.  4757.   153.   116.    17.   234.    92.   190.    22.  5958.]
 [   38.   172.   174.  5150.    31.   122.    59.   122.   135.   128.  6131.]
 [   13.   104.    41.     5.  5189.    52.    45.    24.    60.   309.  5842.]
 [  164.    94.    30.   448.   103.  3974.   185.    44.   237.   142.  5421.]
 [  104.    78.    77.     2.    64.   106.  5448.     0.    36.     3.  5918.]
 [   55.   191.    36.    48.   165.     9.     4.  5443.    13.   301.  6265.]
 [   69.   492.    64.   225.   102.   220.    64.    21.  4417.   177.  5851.]
 [   67.    66.    26.   115.   365.    12.     4.   513.    39.  4742.  5949.]
 [ 6280.  8026.  5262.  6182.  6180.  4588.  6122.  6277.  5247.  5836. 60000.]]


In [13]:
## Calculate the error rates from the confusion matrix
one_v_all_error_rates = calculateErrorRate(one_v_all_confusion)
for i, error_rate in enumerate(one_v_all_error_rates[:-1]):
    print('Error rate for ', i, ': ', error_rate)
else:
    print('Average error rate: ', one_v_all_error_rates[-1])

Error rate for  0 :  0.04288367381394564
Error rate for  1 :  0.029516463957282704
Error rate for  2 :  0.20157771064115476
Error rate for  3 :  0.1600065242211711
Error rate for  4 :  0.11177678877096885
Error rate for  5 :  0.2669249216011806
Error rate for  6 :  0.07941872254139912
Error rate for  7 :  0.13120510774142058
Error rate for  8 :  0.24508631003247308
Error rate for  9 :  0.20289124222558413
Average error rate:  0.14446666666666666


**The one-versus-one classifier identifies the hand-written digits with about 14.4% total error rate on the training data.**

## One-versus-one Classifier on Training Data ##

In [14]:
## Run one-versus-one classifier on the training data
one_v_one_predictions = oneVersusOneClassifier(train_x, train_y)

In [15]:
## Build confusion matrix for the results to the one-versus-one classifier
np.set_printoptions(suppress=True)
np.set_printoptions( linewidth=100)
one_v_one_confusion = buildConfusionMatrix(train_y, one_v_one_predictions)
print(one_v_one_confusion)

[[ 5756.     3.    22.    13.    16.    34.    31.     7.    40.     1.  5923.]
 [    1.  6624.    37.    17.     4.    15.     3.    12.    22.     7.  6742.]
 [   26.    81.  5504.    52.    67.    32.    46.    47.    86.    17.  5958.]
 [   17.    44.   124.  5557.     7.   176.    23.    49.    93.    41.  6131.]
 [   11.    20.    15.     2.  5575.    10.    17.    19.     6.   167.  5842.]
 [   38.    43.    43.   141.    26.  4968.    92.     8.    45.    17.  5421.]
 [   17.    17.    43.     2.    36.    89.  5685.     0.    28.     1.  5918.]
 [    4.    81.    56.     5.    82.    14.     1.  5862.     6.   154.  6265.]
 [   16.   200.    50.   114.    45.   148.    38.    30.  5143.    67.  5851.]
 [   14.    14.    23.    81.   164.    37.     2.   154.    32.  5428.  5949.]
 [ 5900.  7127.  5917.  5984.  6022.  5523.  5938.  6188.  5501.  5900. 60000.]]


In [16]:
## Calculate the error rates from the confusion matrix
one_v_one_error_rates = calculateErrorRate(one_v_one_confusion)
for i, error_rate in enumerate(one_v_one_error_rates[:-1]):
    print('Error rate for ', i, ': ', error_rate)
else:
    print('Average error rate: ', one_v_one_error_rates[-1])

Error rate for  0 :  0.028195171365861894
Error rate for  1 :  0.017502224859092256
Error rate for  2 :  0.07620006713662303
Error rate for  3 :  0.093622573805252
Error rate for  4 :  0.04570352618966107
Error rate for  5 :  0.0835639180962922
Error rate for  6 :  0.039371409259885096
Error rate for  7 :  0.06432561851556265
Error rate for  8 :  0.12100495641770638
Error rate for  9 :  0.08757774415868214
Average error rate:  0.06496666666666667


**The one-versus-one classifier identifies the hand-written digits with about 6.5% total error rate on the training data. This is about half of the error rate of the one-versus-all classifier.**

## One-versus-all Classifier on Testing Data ##

In [17]:
## Run one-versus-all classifier on the testing data
one_v_all_predictions = oneVersusAllClassifier(train_x, train_y, test_x)

In [18]:
## Build confusion matrix for the results to the one-versus-all classifier
np.set_printoptions(suppress=True)
np.set_printoptions( linewidth=100)
one_v_all_confusion = buildConfusionMatrix(test_y, one_v_all_predictions)
print(one_v_all_confusion)

[[  944.     0.     1.     2.     2.     8.    13.     2.     7.     1.   980.]
 [    0.  1107.     2.     2.     3.     1.     5.     1.    14.     0.  1135.]
 [   18.    54.   815.    26.    16.     0.    38.    22.    39.     4.  1032.]
 [    4.    18.    22.   884.     5.    16.    10.    22.    20.     9.  1010.]
 [    0.    22.     6.     0.   883.     3.     9.     1.    12.    46.   982.]
 [   24.    19.     3.    74.    24.   656.    24.    13.    38.    17.   892.]
 [   17.     9.    10.     0.    22.    17.   876.     0.     7.     0.   958.]
 [    5.    43.    14.     6.    25.     1.     1.   883.     1.    49.  1028.]
 [   14.    48.    11.    31.    26.    40.    17.    13.   756.    18.   974.]
 [   16.    10.     3.    17.    80.     0.     1.    75.     4.   803.  1009.]
 [ 1042.  1330.   887.  1042.  1086.   742.   994.  1032.   898.   947. 10000.]]


In [19]:
## Calculate the error rates from the confusion matrix
one_v_all_error_rates = calculateErrorRate(one_v_all_confusion)
for i, error_rate in enumerate(one_v_all_error_rates[:-1]):
    print('Error rate for ', i, ': ', error_rate)
else:
    print('Average error rate: ', one_v_all_error_rates[-1])

Error rate for  0 :  0.036734693877551024
Error rate for  1 :  0.024669603524229075
Error rate for  2 :  0.21027131782945738
Error rate for  3 :  0.12475247524752475
Error rate for  4 :  0.10081466395112017
Error rate for  5 :  0.2645739910313901
Error rate for  6 :  0.08559498956158663
Error rate for  7 :  0.14105058365758755
Error rate for  8 :  0.22381930184804927
Error rate for  9 :  0.20416253716551042
Average error rate:  0.1393


**The one-versus-all classifier identifies the hand-written digits with about 14.0% total error rate on the testing data. This is consistent with its performance on the training data, which means the one-versus-all classifier generalizes on the testing data very well. Among its performances on various digits, "1" is the easiest to recognize with about 2.5% error rate whereas "5" is the hardest to recognize with about 26.5% error rate.**

## One-versus-one Classifier on Testing Data ##

In [20]:
## Run one-versus-one classifier on the testing data
one_v_one_predictions = oneVersusOneClassifier(train_x, train_y, test_x)

In [21]:
## Build confusion matrix for the results to the one-versus-one classifier
np.set_printoptions(suppress=True)
np.set_printoptions( linewidth=100)
one_v_one_confusion = buildConfusionMatrix(test_y, one_v_one_predictions)
print(one_v_one_confusion)

[[  949.     0.     4.     2.     2.     8.     8.     4.     2.     1.   980.]
 [    0.  1122.     3.     3.     1.     1.     2.     1.     2.     0.  1135.]
 [    6.    22.   931.    14.    13.     5.    11.     8.    22.     0.  1032.]
 [    4.     1.    19.   927.     3.    20.     3.    10.    18.     5.  1010.]
 [    0.     2.     7.     1.   934.     3.     6.     3.     2.    24.   982.]
 [    8.     6.     2.    30.     9.   802.    16.     2.    13.     4.   892.]
 [    6.     6.    10.     0.     8.    20.   906.     0.     2.     0.   958.]
 [    1.    17.    17.     3.    10.     1.     0.   955.     2.    22.  1028.]
 [    5.    16.    10.    21.    11.    36.    12.    10.   840.    13.   974.]
 [    4.     5.     1.    10.    31.    12.     1.    23.     5.   917.  1009.]
 [  983.  1197.  1004.  1011.  1022.   908.   965.  1016.   908.   986. 10000.]]


In [22]:
## Calculate the error rates from the confusion matrix
one_v_one_error_rates = calculateErrorRate(one_v_one_confusion)
for i, error_rate in enumerate(one_v_one_error_rates[:-1]):
    print('Error rate for ', i, ': ', error_rate)
else:
    print('Average error rate: ', one_v_one_error_rates[-1])

Error rate for  0 :  0.03163265306122449
Error rate for  1 :  0.01145374449339207
Error rate for  2 :  0.09786821705426356
Error rate for  3 :  0.08217821782178218
Error rate for  4 :  0.048879837067209775
Error rate for  5 :  0.10089686098654709
Error rate for  6 :  0.054279749478079335
Error rate for  7 :  0.07101167315175097
Error rate for  8 :  0.1375770020533881
Error rate for  9 :  0.09117938553022795
Average error rate:  0.0717


**The one-versus-one classifier identifies the hand-written digits with about 7.2% total error rate on the testing data. This is consistent with its performance on the training data, which means the one-versus-one classifier also generalizes well on the testing data. Among its performances on various digits, "1" is the easiest to recognize with about 1.1% error rate and "8" is the hardest to recognize with about 13.8% error rate, which shows that the one-versus-one classifier behaves differently than the one-versus-all classifier.**

**Similar to their performances on the training data, the one-versus-one classifier's total error rate on the testing data is approximately half of the one-versus-all classifier's total error rate on the testing data.**